## Connecting to ClickHouse

In [ ]:
# import clickhouse_connect

# # Connect to ClickHouse
# clickhouse_client = clickhouse_connect.get_client(host='localhost', port=8123)

## Connecting to MySQL

In [ ]:
# import mysql.connector

# # Connect to MySQL
# mysql_conn = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     password="root",
#     database="nyc_taxi"
# )
# mysql_cursor = mysql_conn.cursor()

# Setup

Je sais pas trop comment faire pour le moment.

## Creating ClickHouse Table

In [ ]:
clickhouse_client.command("""
CREATE DATABASE IF NOT EXISTS nyc_taxi;
""")

clickhouse_client.command("""
CREATE TABLE nyc_taxi.trips (
    trip_id Int32,
    pickup_datetime DateTime,
    dropoff_datetime DateTime,
    passenger_count Int32,
    trip_distance Float32,
    fare_amount Float32,
    tip_amount Float32,
    total_amount Float32
) ENGINE = MergeTree()
ORDER BY pickup_datetime;
""")


## Query 1: Total Trips

### MySQL

In [ ]:
mysql_cursor.execute("SELECT COUNT(*) FROM trips;")
print(mysql_cursor.fetchone())

### ClickHouse

In [ ]:
clickhouse_query = "SELECT COUNT(*) FROM nyc_taxi.trips;"
clickhouse_result = clickhouse_client.query(clickhouse_query)
print(clickhouse_result.result_rows)

## Query 2: Total Revenue from All Trips

### MySQL

In [ ]:
mysql_cursor.execute("SELECT SUM(total_amount) FROM trips;")
print(mysql_cursor.fetchone())

### ClickHouse

In [ ]:
clickhouse_query = "SELECT SUM(total_amount) FROM nyc_taxi.trips;"
clickhouse_result = clickhouse_client.query(clickhouse_query)
print(clickhouse_result.result_rows)

## Query 3: Number of Trips per Month

### MySQL

In [ ]:
mysql_cursor.execute("""
SELECT 
    MONTH(pickup_datetime) AS month, 
    COUNT(*) AS trip_count 
FROM trips 
GROUP BY month 
ORDER BY month;
""")
print(mysql_cursor.fetchall())

### ClickHouse

In [ ]:
clickhouse_query = """
SELECT 
    toMonth(pickup_datetime) AS month, 
    COUNT(*) AS trip_count 
FROM nyc_taxi.trips 
GROUP BY month 
ORDER BY month;
"""
clickhouse_result = clickhouse_client.query(clickhouse_query)
print(clickhouse_result.result_rows)

## Query 4: Average Fare per Passenger Count

### MySQL

In [ ]:
mysql_cursor.execute("""
SELECT 
    passenger_count, 
    AVG(fare_amount) AS avg_fare 
FROM trips 
GROUP BY passenger_count 
ORDER BY passenger_count;
""")
print(mysql_cursor.fetchall())

### ClickHouse

In [ ]:
clickhouse_query = """
SELECT 
    passenger_count, 
    AVG(fare_amount) AS avg_fare 
FROM nyc_taxi.trips 
GROUP BY passenger_count 
ORDER BY passenger_count;
"""
clickhouse_result = clickhouse_client.query(clickhouse_query)
print(clickhouse_result.result_rows)

### Comparison of Results